In [3]:
import pandas as pd
import json
import datetime

In [4]:
def decode_v(x):
    r = x[1:-1]
    r = r.replace("'",'"')
    s = r.split(': ')[0]+': '
    r = r.split(s)[1:]
    r = [s+i for i in r]
    r = [i[:-2] if i[-2:]==', ' else i for i in r]
    r = [json.loads(i) for i in r]
    return r

def ts2dt(x):
    return datetime.datetime.fromtimestamp(x)


def get_vout(block_path):
    d = pd.read_csv(block_path)
    d['vout'] = d['vout'].map(decode_v)

    t = enumerate(d['vout'].values)
    t = [{**{'index':i},**j} for (i,v) in t for j in v]
    t = pd.DataFrame.from_dict(t)
    
    vout = t['scriptPubKey'].values
    vout = pd.DataFrame.from_records(vout)
    vout = pd.concat([t.iloc[:,:2], vout], axis=1)
    vout = pd.merge(vout,d.reset_index(), how='left', on='index')
    vout['time'] = vout['time'].map(ts2dt)
    vout['date'] = vout['time'].dt.date
    return vout



def get_vin(block_path):
    d = pd.read_csv(block_path)
    d['vin'] = d['vin'].map(decode_v)

    t = enumerate(d['vin'].values)
    t = [{**{'index':i},**j} for (i,v) in t for j in v]
    t = pd.DataFrame.from_dict(t)

    vin = t['scriptSig'].map(lambda x: x if x==x else {'asm': None, 'hex': None})#.values
    vin = pd.DataFrame.from_records(vin)
    vin = pd.concat([t.iloc[:,:2], vin], axis=1)
    vin = pd.merge(vin,d.reset_index(), how='left', on='index')

    vin['time'] = vin['time'].map(ts2dt)
    vin['date'] = vin['time'].dt.date

    return vin


In [5]:
def filter_tx(tx, check_path):
    check = pd.read_csv(check_path, index_col=0)
    check['ltime'] = pd.to_datetime(check['ltime'])
    check['date'] = check['ltime'].dt.date
    
    tmp = pd.merge(left=tx, right=check, how='inner', on='date')
    
    tmp['delta_time'] = (tmp['time'] - tmp['ltime']) / datetime.timedelta(minutes=1)
    tmp = tmp[tmp['delta_time'].abs()<30]
    tmp = tmp[tmp['delta_time']>0]
    tmp['delta_sum'] = 2*(tmp['value'] - tmp['btcsum']) / (tmp['btcsum'] + tmp['value'])
    tmp = tmp[tmp['delta_sum'] < 0.1]
    tmp = tmp[tmp['delta_sum'] > 0]

    return tmp

In [6]:
current_block_csv = './rpc_block_834577_txs.csv'
check_csv = './check_btc.csv'

filter_tx(get_vout(current_block_csv),check_csv)

,index,value,asm,desc,hex_x,address,type,in_active_chain,txid,hash,...,hex_y,blockhash,confirmations,time,blocktime,date,ltime,btcsum,delta_time,delta_sum


In [7]:
f = pd.read_csv('./filtered_rpc_vouts2.csv').reset_index()
f['ltime'].unique()

array(['2024-03-13 22:35:25', '2024-03-13 21:50:27',
       '2024-03-13 22:06:34', '2024-03-13 22:08:39',
       '2024-03-14 11:21:29', '2024-03-14 11:36:48',
       '2024-03-16 09:45:33', '2024-03-17 07:52:21'], dtype=object)

In [8]:
cols = ['value', 'asm', 'address', 'txid', 'hash','vin', 'vout', 'blockhash', 'blocktime',
       'blockNumber','ltime', 'btcsum', 'delta_time', 'delta_sum']
#f[cols].to_csv('./bitcoin_nw_res1_raw.csv')

In [21]:
btcl = ['filtered_rpc_vouts_835530_835536_2024_03_20_11_43_54.csv',
        'filtered_rpc_vouts_835542_835548_2024_03_20_14_22_59.csv',
        'filtered_rpc_vouts_836104_836110_2024_03_24_13_52_48.csv',
        'filtered_rpc_vouts_836109_836115_2024_03_24_14_24_07.csv',
        'filtered_rpc_vouts_836242_836248_2024_03_25_13_31_22.csv']

d = ['./pb 2024-06-19/'+btcl[i] for i in range(5)]
d = [pd.read_csv(i) for i in d]

[(i['time'].min(),
  i['time'].max()) for i in d]

[('2024-03-20 11:14:41', '2024-03-20 11:15:53'),
 ('2024-03-20 13:33:43', '2024-03-20 13:33:43'),
 (nan, nan),
 ('2024-03-24 13:56:02', '2024-03-24 13:56:02'),
 ('2024-03-25 12:03:39', '2024-03-25 12:03:39')]

In [17]:
d = ['./pb 2024-06-19/'+btcl[i] for i in range(5)]
d = [pd.read_csv(i) for i in d]
d = pd.concat(d)
d['vout'].map(decode_v).map(lambda x: sum([i['value']==0.000713040 for i in x])).sum()
d['txid'].isin(['b8bea467a95c2a2358540190f6654856d11412e8304e68714a88bc430cd3beff']).sum()

/tmp/ipykernel_336273/3461870833.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  d = pd.concat(d)


0

In [18]:
d.T

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
Unnamed: 0,1464,2316,6671,7573,7575,7579,8491,10310,10372,10506,...,38022,41932,42782,43972,45127,45434,48494,57368,58693,59747
index,17,27,78,89,89,89,101,126,126,127,...,838,886,886,886,892,898,921,979,983,983
value,0.000266,0.000344,0.000345,0.000053,0.000053,0.000053,0.000345,0.000389,0.001853,0.000344,...,0.000165,0.000164,0.000167,0.000167,0.000202,0.000169,0.000159,0.000167,0.000211,0.000209
asm,1 af8fef224377442e2a723a84a87b41ec8f97bec0d7f5...,1 aa741b4dd9867247e941691b9dd704b00f5ff77cf739...,1 fb95a9d4e372ba9b299c3423999b2ecabeb3d3eb7bbd...,1 79501b23fc72d95c1bfff5e3b0c391114471166e3692...,1 79501b23fc72d95c1bfff5e3b0c391114471166e3692...,1 79501b23fc72d95c1bfff5e3b0c391114471166e3692...,1 c77478e732411917ee1848143f2e92f04af8efcb6321...,1 03f9b8af8c21f3ad03ab0564303b3bf026fe396181fd...,1 8b9bc35537905d680f1705cb488e62269cfd14700d82...,1 d9da4f54f374ec97cf12e4eec7751786a9c75269cc2b...,...,1 6c2e3f96d69a5b080c7f58b156ca698f37a2c63e467a...,0 ab28fa60075ceaad17286538e0d4296694134e77,0 fe9fb13d7527bf8ff13a105ee21133e4c4ec22de,0 42f05e9b3323e9fbc75ac9645a06edf475f1b28a,0 fdda11396644386f646e056be9fb1c840fbc52e5,0 9b03002b40d73a6e0493148c0b5b5881df23e910,OP_HASH160 edbf32753255219d3170ca9b004630399f5...,0 44177ae95949f440055e03f45453dda78242c850,OP_DUP OP_HASH160 e0f1575a129d721b4dea66d7050b...,0 a5a814ebacd9627cd2abc599a92e04128c3aa7ba
desc,rawtr(af8fef224377442e2a723a84a87b41ec8f97bec0...,rawtr(aa741b4dd9867247e941691b9dd704b00f5ff77c...,rawtr(fb95a9d4e372ba9b299c3423999b2ecabeb3d3eb...,rawtr(79501b23fc72d95c1bfff5e3b0c391114471166e...,rawtr(79501b23fc72d95c1bfff5e3b0c391114471166e...,rawtr(79501b23fc72d95c1bfff5e3b0c391114471166e...,rawtr(c77478e732411917ee1848143f2e92f04af8efcb...,rawtr(03f9b8af8c21f3ad03ab0564303b3bf026fe3961...,rawtr(8b9bc35537905d680f1705cb488e62269cfd1470...,rawtr(d9da4f54f374ec97cf12e4eec7751786a9c75269...,...,rawtr(6c2e3f96d69a5b080c7f58b156ca698f37a2c63e...,addr(bc1q4v505cq8tn4269egv5uwp4pfv62pxnnhseuey...,addr(bc1ql60mz0t4y7lcluf6zp0wyyfnunzwcgk7fr7a5...,addr(bc1qgtc9axeny05lh366e9j95phd736lrv52etr4a...,addr(bc1qlhdpzwtxgsux7erwq447n7cuss8mc5h9waxg7...,addr(bc1qnvpsq26q6uaxupynzjxqkk6cs80j86gsglpv0...,addr(3PN77cdBUFJxMKJe8sjxzkW7ZrJ7GAqoQK)#r77973hp,addr(bc1qgsth462ef86yqp27q069g57a57py9jzs9n8fm...,addr(1MWPUeok3Vnwq5kMCkHUkwWmoRhzrVE78J)#vasq72w7,addr(bc1q5k5pf6avm938e54tckv6jtsyz2xr4fa6p6nsn...
hex_x,5120af8fef224377442e2a723a84a87b41ec8f97bec0d7...,5120aa741b4dd9867247e941691b9dd704b00f5ff77cf7...,5120fb95a9d4e372ba9b299c3423999b2ecabeb3d3eb7b...,512079501b23fc72d95c1bfff5e3b0c391114471166e36...,512079501b23fc72d95c1bfff5e3b0c391114471166e36...,512079501b23fc72d95c1bfff5e3b0c391114471166e36...,5120c77478e732411917ee1848143f2e92f04af8efcb63...,512003f9b8af8c21f3ad03ab0564303b3bf026fe396181...,51208b9bc35537905d680f1705cb488e62269cfd14700d...,5120d9da4f54f374ec97cf12e4eec7751786a9c75269cc...,...,51206c2e3f96d69a5b080c7f58b156ca698f37a2c63e46...,0014ab28fa60075ceaad17286538e0d4296694134e77,0014fe9fb13d7527bf8ff13a105ee21133e4c4ec22de,001442f05e9b3323e9fbc75ac9645a06edf475f1b28a,0014fdda11396644386f646e056be9fb1c840fbc52e5,00149b03002b40d73a6e0493148c0b5b5881df23e910,a914edbf32753255219d3170ca9b004630399f5a5a6c87,001444177ae95949f440055e03f45453dda78242c850,76a914e0f1575a129d721b4dea66d7050ba20d8e5b7a1f...,0014a5a814ebacd9627cd2abc599a92e04128c3aa7ba
address,bc1p47877gjrwazzu2nj82z2s76paj8e00kq6l65xdagmn...,bc1p4f6pknweseey062pdydem4cykq84lamu7uukekyh2n...,bc1plw26n48rw2afk2vuxs3enxewe2lt85lt0w7kx3z2u3...,bc1p09gpkgluwtv4cxll7h3mpsu3z9z8z9nwx6fx200pk4...,bc1p09gpkgluwtv4cxll7h3mpsu3z9z8z9nwx6fx200pk4...,bc1p09gpkgluwtv4cxll7h3mpsu3z9z8z9nwx6fx200pk4...,bc1pca683eejgyv30mscfq2r7t5j7p903m7tvvslsyg467...,bc1pq0um3tuvy8e66qatq4jrqwem7qn0uwtps87nk7skz7...,bc1p3wdux4fhjpwksrchqh953rnzy6w069rspkpfrahxly...,bc1pm8dy748nwnkf0ncjunhvwaghs65uw5nfes4uh68h2n...,...,bc1pdshrl9kknfdssrrltzc4djnf3um69337geaqurk37n...,bc1q4v505cq8tn4269egv5uwp4pfv62pxnnhseuey

In [14]:
i = 3
pd.read_csv('./pb 2024-06-19/'+btcl[i])['time']

0     2024-03-24 13:56:02
1     2024-03-24 13:56:02
2     2024-03-24 13:56:02
3     2024-03-24 13:56:02
4     2024-03-24 13:56:02
5     2024-03-24 13:56:02
6     2024-03-24 13:56:02
7     2024-03-24 13:56:02
8     2024-03-24 13:56:02
9     2024-03-24 13:56:02
10    2024-03-24 13:56:02
11    2024-03-24 13:56:02
12    2024-03-24 13:56:02
13    2024-03-24 13:56:02
14    2024-03-24 13:56:02
15    2024-03-24 13:56:02
16    2024-03-24 13:56:02
17    2024-03-24 13:56:02
18    2024-03-24 13:56:02
19    2024-03-24 13:56:02
20    2024-03-24 13:56:02
21    2024-03-24 13:56:02
22    2024-03-24 13:56:02
23    2024-03-24 13:56:02
24    2024-03-24 13:56:02
25    2024-03-24 13:56:02
26    2024-03-24 13:56:02
27    2024-03-24 13:56:02
28    2024-03-24 13:56:02
29    2024-03-24 13:56:02
30    2024-03-24 13:56:02
Name: time, dtype: object